In [1]:
import sys
import time
import bibtexparser
import itertools
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
import bibtexparser
bibtex_file = open('gap-publishednicer.bib.txt', encoding='utf-8')
bib_data = bibtexparser.load(bibtex_file)

In [5]:
bib = bib_data.entries

In [10]:
final_df = pd.read_csv('local_test_output.csv', dtype='str')

In [15]:
final_df

,MR,Citation
0,MR4044696,"The GAP Group, GAP – groups, algorithms and pr..."
1,MR2900886,"The GAP Group, $GAP$ groups, algorithms, and p..."
2,MR3169623,"Distler, A., Mitchell, J. D. (2011). Smallsemi..."
3,MR3169623,"The GAP Group, (2008). (http://www.gap-system...."
4,MR4180136,"The GAP Group, 2019. GAP – Groups, Algorithms,..."


In [85]:
mrs = final_df['MR']

In [91]:
mrs[1][0:2]

'MR'

In [12]:
print(type(bib))
print(type(bib[5]))
print(bib[5])

<class 'list'>
<class 'dict'>
{'printedkey': 'A01', 'mrreviewer': 'H. Heineken', 'mrnumber': '1864795', 'mrclass': '20F45 (20D60 20F19)', 'issn': '0362-1588', 'fjournal': 'Houston Journal of Mathematics', 'pages': '511–522', 'year': '2001', 'number': '3', 'volume': '27', 'journal': 'Houston J. Math.', 'title': 'Some {E}ngel conditions on finite subsets of certain groups', 'author': 'Abdollahi, A.', 'ENTRYTYPE': 'article', 'ID': 'MR1864795'}


In [13]:
bib[5]

{'printedkey': 'A01',
 'mrreviewer': 'H. Heineken',
 'mrnumber': '1864795',
 'mrclass': '20F45 (20D60 20F19)',
 'issn': '0362-1588',
 'fjournal': 'Houston Journal of Mathematics',
 'pages': '511–522',
 'year': '2001',
 'number': '3',
 'volume': '27',
 'journal': 'Houston J. Math.',
 'title': 'Some {E}ngel conditions on finite subsets of certain groups',
 'author': 'Abdollahi, A.',
 'ENTRYTYPE': 'article',
 'ID': 'MR1864795'}

In [14]:
bib[5].keys()
bib[5]['year']
bib[5]['fjournal']

'Houston Journal of Mathematics'

In [79]:
df = pd.DataFrame(bib)


In [82]:
df_filtered = df[df.mrnumber.isin(mrs)]

In [83]:
df_filtered

,printedkey,doi,url,mrreviewer,mrnumber,mrclass,issn,fjournal,pages,year,...,annote,type,address,institution,howpublished,editor,bookeditor,edition,key,organization


In [65]:
bib_mr = ['MR' + d['mrnumber'] for d in bib if 'mrnumber' in d]
journal = [d['fjournal'] for d in bib if 'fjournal' in d]
year =  [d['year'] for d in bib if 'year' in d]
pub_type =  [d['ENTRYTYPE'] for d in bib if 'ENTRYTYPE' in d]
author = [d['author'] for d in bib if 'author' in d]

In [77]:
for i in bib if mrs in bib_mr
bib[i]

SyntaxError: invalid syntax (<ipython-input-77-b3794065cff7>, line 1)

In [67]:
print(type(bib_mr))
print(bib_mr[0:5])

<class 'list'>
['MR4056124', 'MR3942387', 'MR3354065', 'MR3646312', 'MR1864795']


In [88]:
targets = ['A01', 'A03', 'A05']

lis_dic = [{'ID': 'A01',
 'man': 'H. Heineken',
 'year': '1999'},
{'ID': 'A02',
 'man': 'H. Henkel',
 'year': '1555'},
{'ID': 'A03',
 'man': 'H. Stenkel',
 'year': '1597'},
{'ID': 'A04',
 'man': 'H. Stanzel',
 'year': '1999'},
{'ID': 'A05',
 'man': 'H. Panzel',
 'year': '1333'}]

In [89]:
df = pd.DataFrame(lis_dic)
df = df[df.ID.isin(targets)]

In [90]:
df

,ID,man,year
0,A01,H. Heineken,1999
2,A03,H. Stenkel,1597
4,A05,H. Panzel,1333
